In [1]:
import sys

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
sys.path.append("../utils/")
sys.path.append("../")

In [5]:
import metrics

In [6]:
from agg_crossfit_results import *

In [7]:
pd.set_option('display.max_rows', 500)


In [8]:
base_dir="../runs/pretrained/1e-4/0/"

In [9]:
set_name = 'third_set'

dir_dict = {"BART-BiHNet+Reg":f"../runs/BiHNet_Reg_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+EWC":f"../runs/BiHNet_ewc_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Multitask": f"../runs/adpter_mtl_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet-Multitask": f"../runs/mtl_hnet_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-Adapter-Vanilla": f"../runs/adapter_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
            "BART-BiHNet+Vanilla": f"../runs/BiHNet_vanilla_{set_name}_0.01_s64_d256_limit/1e-4/0/",
           }

In [10]:
dir_dict

{'BART-BiHNet+Reg': '../runs/BiHNet_Reg_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet+EWC': '../runs/BiHNet_ewc_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-Adapter-Multitask': '../runs/adpter_mtl_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet-Multitask': '../runs/mtl_hnet_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-Adapter-Vanilla': '../runs/adapter_vanilla_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet+Vanilla': '../runs/BiHNet_vanilla_third_set_0.01_s64_d256_limit/1e-4/0/'}

In [12]:
df = make_few_shot_result_table(dir_dict["BART-Adapter-Vanilla"], postfix="_naive_16shot", metrics_type='emf1', with_dash=False)

results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv


In [17]:
dfs = []
for alg in dir_dict:
    print(alg)
    for with_dash in [True, False]:
        df =  make_few_shot_result_table(dir_dict[alg], postfix="_naive_16shot", metrics_type='emf1', with_dash=with_dash)
        if df.empty:
            print("df", df)
            print(alg, with_dash, "hereeee")
            continue
        if "task_detail" in df.columns:
            df = pd.DataFrame(df.groupby(["task_name", "task_detail"])["nf1"].max()).reset_index()
        else:
            df = pd.DataFrame(df.groupby(["task_name"])["nf1"].max()).reset_index()
        df["alg"] = alg
        dfs.append(df)

MTL
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
CL
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv
Vanilla
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+)_fewshot_at_-1_naive_16shot.csv


In [21]:
all_fs_df = pd.concat(dfs)

In [59]:
all_fs_df=all_fs_df[all_fs_df.task_detail != "west"]

In [22]:
all_fs_df["nf1"] = all_fs_df["nf1"].map('{:.3f}'.format)
all_fs_df=all_fs_df.fillna("singlelabel")

In [23]:
latex_table = all_fs_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)

In [24]:
# all_fs_df = all_fs_df[all_fs_df['task_name'] != 'dygen']

In [25]:
print(latex_table)

\begin{tabular}{llll}
\toprule
task_name & task_detail & nf1 & alg \\
\midrule
BAD4 & singlelabel & 0.446 & CL \\
BAD4 & singlelabel & 0.439 & MTL \\
BAD4 & singlelabel & 0.429 & Vanilla \\
cmsb & sexist & 0.480 & CL \\
cmsb & sexist & 0.498 & MTL \\
cmsb & sexist & 0.513 & Vanilla \\
conan & poc & 0.773 & CL \\
conan & poc & 0.508 & MTL \\
conan & poc & 0.415 & Vanilla \\
misogyny & singlelabel & 0.352 & CL \\
misogyny & singlelabel & 0.321 & MTL \\
misogyny & singlelabel & 0.320 & Vanilla \\
stormfront & singlelabel & 0.827 & CL \\
stormfront & singlelabel & 0.833 & MTL \\
stormfront & singlelabel & 0.852 & Vanilla \\
us_election & hof & 0.208 & CL \\
us_election & hof & 0.313 & MTL \\
us_election & hof & 0.341 & Vanilla \\
\bottomrule
\end{tabular}



In [26]:
new_string = latex_table.replace("Vanilla \\\\", "Vanilla \\\\ \n \hline")
new_string =  new_string.replace("_", "--")

In [27]:
print(new_string)

\begin{tabular}{llll}
\toprule
task--name & task--detail & nf1 & alg \\
\midrule
BAD4 & singlelabel & 0.446 & CL \\
BAD4 & singlelabel & 0.439 & MTL \\
BAD4 & singlelabel & 0.429 & Vanilla \\ 
 \hline
cmsb & sexist & 0.480 & CL \\
cmsb & sexist & 0.498 & MTL \\
cmsb & sexist & 0.513 & Vanilla \\ 
 \hline
conan & poc & 0.773 & CL \\
conan & poc & 0.508 & MTL \\
conan & poc & 0.415 & Vanilla \\ 
 \hline
misogyny & singlelabel & 0.352 & CL \\
misogyny & singlelabel & 0.321 & MTL \\
misogyny & singlelabel & 0.320 & Vanilla \\ 
 \hline
stormfront & singlelabel & 0.827 & CL \\
stormfront & singlelabel & 0.833 & MTL \\
stormfront & singlelabel & 0.852 & Vanilla \\ 
 \hline
us--election & hof & 0.208 & CL \\
us--election & hof & 0.313 & MTL \\
us--election & hof & 0.341 & Vanilla \\ 
 \hline
\bottomrule
\end{tabular}



In [28]:
print(all_fs_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False))

\begin{tabular}{llll}
\toprule
task_name & task_detail & nf1 & alg \\
\midrule
BAD4 & singlelabel & 0.446 & CL \\
BAD4 & singlelabel & 0.439 & MTL \\
BAD4 & singlelabel & 0.429 & Vanilla \\
cmsb & sexist & 0.480 & CL \\
cmsb & sexist & 0.498 & MTL \\
cmsb & sexist & 0.513 & Vanilla \\
conan & poc & 0.773 & CL \\
conan & poc & 0.508 & MTL \\
conan & poc & 0.415 & Vanilla \\
misogyny & singlelabel & 0.352 & CL \\
misogyny & singlelabel & 0.321 & MTL \\
misogyny & singlelabel & 0.320 & Vanilla \\
stormfront & singlelabel & 0.827 & CL \\
stormfront & singlelabel & 0.833 & MTL \\
stormfront & singlelabel & 0.852 & Vanilla \\
us_election & hof & 0.208 & CL \\
us_election & hof & 0.313 & MTL \\
us_election & hof & 0.341 & Vanilla \\
\bottomrule
\end{tabular}



In [29]:
for _,group in all_fs_df.groupby(["task_name", "task_detail"]):
    print(pd.DataFrame(group))
    print("-----")

  task_name  task_detail    nf1      alg
0      BAD4  singlelabel  0.439      MTL
0      BAD4  singlelabel  0.446       CL
0      BAD4  singlelabel  0.429  Vanilla
-----
  task_name task_detail    nf1      alg
0      cmsb      sexist  0.498      MTL
0      cmsb      sexist  0.480       CL
0      cmsb      sexist  0.513  Vanilla
-----
  task_name task_detail    nf1      alg
1     conan         poc  0.508      MTL
1     conan         poc  0.773       CL
1     conan         poc  0.415  Vanilla
-----
  task_name  task_detail    nf1      alg
1  misogyny  singlelabel  0.321      MTL
1  misogyny  singlelabel  0.352       CL
1  misogyny  singlelabel  0.320  Vanilla
-----
    task_name  task_detail    nf1      alg
2  stormfront  singlelabel  0.833      MTL
2  stormfront  singlelabel  0.827       CL
2  stormfront  singlelabel  0.852  Vanilla
-----
     task_name task_detail    nf1      alg
2  us_election         hof  0.313      MTL
2  us_election         hof  0.208       CL
2  us_election       

In [30]:
all_fs_df

,task_name,task_detail,nf1,alg
0,cmsb,sexist,0.498,MTL
1,conan,poc,0.508,MTL
2,us_election,hof,0.313,MTL
0,BAD4,singlelabel,0.439,MTL
1,misogyny,singlelabel,0.321,MTL
2,stormfront,singlelabel,0.833,MTL
0,cmsb,sexist,0.480,CL
1,conan,poc,0.773,CL
2,us_election,hof,0.208,CL
0,BAD4,singlelabel,0.446,CL


In [31]:
all_fs_df["nf1"] = all_fs_df["nf1"].astype(float)

In [32]:
all_fs_df.groupby("alg")["nf1"].mean()

alg
CL         0.514333
MTL        0.485333
Vanilla    0.478333
Name: nf1, dtype: float64

In [33]:
pd.DataFrame(df.groupby(["task_name", "task_detail"])["nf1"].max()).reset_index()

KeyError: 'task_detail'

,task_name,task_detail,nf1
0,cmsb,sexist,0.877566
1,ucc,antagonize,0.952500
2,ucc,condescending,0.940000
3,ucc,dismissive,0.966875
4,ucc,generalisation,0.980625
5,ucc,generalisation_unfair,0.982500
6,ucc,healthy,0.935625
7,ucc,hostile,0.977500
8,ucc,sarcastic,0.953750
9,us_election,biden,0.509979


In [23]:
idx

0      0.491497
1      0.896667
2      0.877566
3      0.491497
4      0.966875
         ...   
99     0.982500
100    0.896667
101    0.982500
102    0.952500
103    1.000000
Name: nf1, Length: 104, dtype: float64

In [83]:
pattern = "results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_(\d+)_fewshot_at_-1{}.csv".format("_hnet_16shot")

In [86]:
new_df = process_few_shot_df(df)

In [87]:
new_df

,task_detail,step,nf1
0,antagonize,200,0.456875
1,antagonize,400,0.952500
2,antagonize,600,0.920000
3,antagonize,800,0.951250
4,antagonize,1000,0.946250
...,...,...,...
99,west,800,1.000000
100,west,1000,1.000000
101,west,1200,1.000000
102,west,1400,1.000000


# Final Accuracy

In [11]:
metrics_type = 'auc'
dir_dict

{'BART-BiHNet+Reg': '../runs/BiHNet_Reg_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet+EWC': '../runs/BiHNet_ewc_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-Adapter-Multitask': '../runs/adpter_mtl_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet-Multitask': '../runs/mtl_hnet_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-Adapter-Vanilla': '../runs/adapter_vanilla_third_set_0.01_s64_d256_limit/1e-4/0/',
 'BART-BiHNet+Vanilla': '../runs/BiHNet_vanilla_third_set_0.01_s64_d256_limit/1e-4/0/'}

In [12]:
dfs = []
for alg in dir_dict:
    if alg =="pretrained" :
        continue
    print(alg)
    df1 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=True)
    df2 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=False)
    df = pd.concat([df1, df2])
    df['task_id'] = df['task_id'].astype(int)
    df['at_task'] = df['at_task'].astype(int)
    df['step'] = df['step'].astype(int)
    df = df[df.at_task == max(df.at_task)]
    if "task_detail" in  df.columns:
        idx = df.groupby(["task_name", "task_detail"])["step"].transform(max) == df["step"]
    else:
        print("here")
        idx = df.groupby(["task_name"])["step"].transform(max) == df["step"]

    df = df[idx]                      
    df["alg"] = alg
    dfs.append(df)

BART-BiHNet+Reg
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Adapter-Multitask
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet-Multitask
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Ad

In [13]:
all_final_df = pd.concat(dfs)

In [14]:
all_final_df

,task_id,task_name,task_detail,at_task,step,path,auc,maj,alg
187,0,jigsaw,obscene,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.946478,0.995400,BART-BiHNet+Reg
188,1,ucc,generalisation_unfair,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.682753,0.980122,BART-BiHNet+Reg
198,10,jigsaw,toxicity,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.918422,0.922100,BART-BiHNet+Reg
199,11,personal_attack,tpa,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.841588,0.990371,BART-BiHNet+Reg
200,12,cad,affiliationdirectedabuse,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.643099,0.945647,BART-BiHNet+Reg
201,13,ucc,generalisation,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.685203,0.978315,BART-BiHNet+Reg
196,14,ghc,hd,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.780278,0.920145,BART-BiHNet+Reg
202,15,hate,offensive,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.955545,0.876065,BART-BiHNet+Reg
203,16,abusive,hateful,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.666253,0.942422,BART-BiHNet+Reg
204,17,ucc,dismissive,25,2669,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.681038,0.967698,BART-BiHNet+Reg


In [15]:
all_final_df = all_final_df[[metrics_type, "task_name", "task_detail",  "alg"]]

In [16]:
for _,group in all_final_df.groupby(["task_name", "task_detail"]):
    print(pd.DataFrame(group))
    print("--"*20)

          auc task_name task_detail                     alg
210  0.966972   abusive     abusive         BART-BiHNet+Reg
194  0.681150   abusive     abusive         BART-BiHNet+EWC
179  0.974823   abusive     abusive  BART-Adapter-Multitask
179  0.940765   abusive     abusive   BART-BiHNet-Multitask
215  0.637408   abusive     abusive    BART-Adapter-Vanilla
218  0.784075   abusive     abusive     BART-BiHNet+Vanilla
----------------------------------------
          auc task_name task_detail                     alg
203  0.666253   abusive     hateful         BART-BiHNet+Reg
187  0.728667   abusive     hateful         BART-BiHNet+EWC
172  0.843342   abusive     hateful  BART-Adapter-Multitask
172  0.774595   abusive     hateful   BART-BiHNet-Multitask
208  0.763553   abusive     hateful    BART-Adapter-Vanilla
211  0.770081   abusive     hateful     BART-BiHNet+Vanilla
----------------------------------------
          auc task_name               task_detail                     alg
200 

In [17]:
all_final_df.groupby("alg")[metrics_type].mean()

alg
BART-Adapter-Multitask    0.875212
BART-Adapter-Vanilla      0.678388
BART-BiHNet+EWC           0.657128
BART-BiHNet+Reg           0.785940
BART-BiHNet+Vanilla       0.711536
BART-BiHNet-Multitask     0.832124
Name: auc, dtype: float64

## Instant Accuracy

In [18]:
dfs = []
for alg in dir_dict:
    if alg =="pretrained" or 'Multitask' in alg:
        continue
    print(alg)
    df1 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=True)
    df2 =  make_results_table(dir_dict[alg], metrics_type=metrics_type, with_dash=False)
    df = pd.concat([df1, df2])
    df['task_id'] = df['task_id'].astype(int)
    df['at_task'] = df['at_task'].astype(int)
    df['step'] = df['step'].astype(int)
    df = df[df.task_id == df.at_task]
    if "task_detail" in  df.columns:
        # if df.groupby(["task_name", "task_detail"])["em"].transform(max) == 0.0:
        #     print("here")
        #     idx = df.groupby(["task_name", "task_detail"])["em"].transform(max)
        idx = df.groupby(["task_name", "task_detail"])[metrics_type].transform(max) == df[metrics_type]
    else:
        print("here")
        idx = df.groupby(["task_name"])["em"].transform(max) == df["em"]

    df = df[idx]                      
    df["alg"] = alg
    dfs.append(df)


BART-BiHNet+Reg
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+EWC
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-Adapter-Vanilla
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv
BART-BiHNet+Vanilla
results_task_(\d+)_(\w+)-(\w+)_task_(\d+)_step_([\d]+).csv
Index(['task_id', 'task_name', 'task_detail', 'at_task', 'step', 'path', 'auc',
       'maj'],
      dtype='object')
results_task_(\d+)_(\w+)_task_(\d+)_step_([\d]+).csv


In [19]:
all_instant_df = pd.concat(dfs)

In [20]:
all_instant_df

,task_id,task_name,task_detail,at_task,step,path,auc,maj,alg
6,0,jigsaw,obscene,0,1099,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.978208,0.995400,BART-BiHNet+Reg
12,1,ucc,generalisation_unfair,1,314,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.860750,0.980122,BART-BiHNet+Reg
69,10,jigsaw,toxicity,10,628,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.930403,0.922100,BART-BiHNet+Reg
71,11,personal_attack,tpa,11,314,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.934629,0.990371,BART-BiHNet+Reg
91,12,cad,affiliationdirectedabuse,12,942,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.879390,0.945647,BART-BiHNet+Reg
93,13,ucc,generalisation,13,157,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.875159,0.978315,BART-BiHNet+Reg
100,14,ghc,hd,14,628,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.872039,0.920145,BART-BiHNet+Reg
107,15,hate,offensive,15,1099,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.979944,0.876065,BART-BiHNet+Reg
112,16,abusive,hateful,16,785,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.864749,0.942422,BART-BiHNet+Reg
118,17,ucc,dismissive,17,785,../runs/BiHNet_Reg_third_set_0.01_s64_d256_lim...,0.819899,0.967698,BART-BiHNet+Reg


In [21]:
all_instant_df = all_instant_df[[metrics_type, "task_name", "task_detail",  "alg"]]
# all_instant_df= all_instant_df.fillna("singlelabel")
for _,group in all_instant_df.groupby(["task_name", "task_detail"]):
    print(pd.DataFrame(group))

          auc task_name task_detail                   alg
158  0.972513   abusive     abusive       BART-BiHNet+Reg
155  0.975408   abusive     abusive       BART-BiHNet+EWC
171  0.975141   abusive     abusive  BART-Adapter-Vanilla
173  0.974635   abusive     abusive   BART-BiHNet+Vanilla
          auc task_name task_detail                   alg
112  0.864749   abusive     hateful       BART-BiHNet+Reg
113  0.849809   abusive     hateful       BART-BiHNet+EWC
117  0.858683   abusive     hateful  BART-Adapter-Vanilla
124  0.854976   abusive     hateful   BART-BiHNet+Vanilla
         auc task_name               task_detail                   alg
91  0.879390       cad  affiliationdirectedabuse       BART-BiHNet+Reg
86  0.908008       cad  affiliationdirectedabuse       BART-BiHNet+EWC
91  0.887943       cad  affiliationdirectedabuse  BART-Adapter-Vanilla
97  0.888610       cad  affiliationdirectedabuse   BART-BiHNet+Vanilla
          auc task_name            task_detail                   

In [22]:
all_instant_df.groupby("alg")[metrics_type].mean()

alg
BART-Adapter-Vanilla    0.885862
BART-BiHNet+EWC         0.886328
BART-BiHNet+Reg         0.884639
BART-BiHNet+Vanilla     0.883828
Name: auc, dtype: float64

# LATEX 

In [29]:
latex_table = all_instant_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)
new_string = latex_table.replace("BART-BiHNet+Vanilla \\\\", "BART-BiHNet+Vanilla\\\\ \n \hline")
# new_string =  new_string.replace("_", "--")

In [30]:
print(new_string)

\begin{tabular}{rlll}
\toprule
auc & task_name & task_detail & alg \\
\midrule
0.975141 & abusive & abusive & BART-Adapter-Vanilla \\
0.975408 & abusive & abusive & BART-BiHNet+EWC \\
0.972513 & abusive & abusive & BART-BiHNet+Reg \\
0.974635 & abusive & abusive & BART-BiHNet+Vanilla\\ 
 \hline
0.858683 & abusive & hateful & BART-Adapter-Vanilla \\
0.849809 & abusive & hateful & BART-BiHNet+EWC \\
0.864749 & abusive & hateful & BART-BiHNet+Reg \\
0.854976 & abusive & hateful & BART-BiHNet+Vanilla\\ 
 \hline
0.887943 & cad & affiliationdirectedabuse & BART-Adapter-Vanilla \\
0.908008 & cad & affiliationdirectedabuse & BART-BiHNet+EWC \\
0.879390 & cad & affiliationdirectedabuse & BART-BiHNet+Reg \\
0.888610 & cad & affiliationdirectedabuse & BART-BiHNet+Vanilla\\ 
 \hline
0.780956 & cad & identitydirectedabuse & BART-Adapter-Vanilla \\
0.802461 & cad & identitydirectedabuse & BART-BiHNet+EWC \\
0.799686 & cad & identitydirectedabuse & BART-BiHNet+Reg \\
0.800906 & cad & identitydirected

In [31]:
latex_table = all_final_df.sort_values(by=["task_name", "task_detail", "alg"]).to_latex(index=False)
new_string = latex_table.replace("BART-BiHNet-Multitask \\\\", "BART-BiHNet-Multitask \\\\ \n \hline")
print(new_string)

\begin{tabular}{rlll}
\toprule
auc & task_name & task_detail & alg \\
\midrule
0.974823 & abusive & abusive & BART-Adapter-Multitask \\
0.637408 & abusive & abusive & BART-Adapter-Vanilla \\
0.681150 & abusive & abusive & BART-BiHNet+EWC \\
0.966972 & abusive & abusive & BART-BiHNet+Reg \\
0.784075 & abusive & abusive & BART-BiHNet+Vanilla \\
0.940765 & abusive & abusive & BART-BiHNet-Multitask \\ 
 \hline
0.843342 & abusive & hateful & BART-Adapter-Multitask \\
0.763553 & abusive & hateful & BART-Adapter-Vanilla \\
0.728667 & abusive & hateful & BART-BiHNet+EWC \\
0.666253 & abusive & hateful & BART-BiHNet+Reg \\
0.770081 & abusive & hateful & BART-BiHNet+Vanilla \\
0.774595 & abusive & hateful & BART-BiHNet-Multitask \\ 
 \hline
0.878271 & cad & affiliationdirectedabuse & BART-Adapter-Multitask \\
0.618436 & cad & affiliationdirectedabuse & BART-Adapter-Vanilla \\
0.550398 & cad & affiliationdirectedabuse & BART-BiHNet+EWC \\
0.643099 & cad & affiliationdirectedabuse & BART-BiHNet+Re